## Scott Berry - Final Project Chatbot

Install Chatterbot and spaCy libraries as well as other dependencies

In [36]:
## !pip install chatterbot
## !pip install chatterbot_corpus
## !pip install spacy
## !pip install pytz
## !python -m spacy download en_core_web_sm

Initialize chatbot

In [38]:
from chatterbot import ChatBot

bot = ChatBot(
    "Scorberr_Chatbot",
    logic_adapters=[
        {
            "import_path": "chatterbot.logic.BestMatch",
            "default_response": "Sorry, could you rephrase that? I didn't understand.",
            "maximum_similarity_threshold": 0.90
        }
    ],
    preprocessors=[
        "chatterbot.preprocessors.clean_whitespace",
        "chatterbot.preprocessors.unescape_html",
        "chatterbot.preprocessors.convert_to_ascii"
    ]
)

ModuleNotFoundError: No module named 'chatterbot.ext'

Train chatbot

In [ ]:
from chatterbot.trainers import ListTrainer
trainer = ListTrainer(bot)

trainer.train([
    "Hi, how can I assist?",
    "Hello, what temperature does water boil at?",
    "Water boils at 212 degrees Fahrenheit and 100 degrees Celsius.",
    "Okay, thank you!",
    "You're welcome, have a good day!"
])

Test Chatbot

In [ ]:
response = bot.get_response("water boiling temperature")
print(response)

Create a while loop for chatbot activity

In [ ]:
name = input("Enter your name: ")

print("Welcome to Scorberr\\'s chatbot, how can I help?")

while True:

    request = input(name + ":")

    if request.lower() == "bye":
        print("Bot: Bye")
        break
    else:
        response = bot.get_response(request)
        print("Bot: ", response)

Train on chatterbot corpus library

In [ ]:
from chatterbot.trainers import ChatterBotCorpusTrainer

trainer = ChatterBotCorpusTrainer(bot)

corpus_library = [
    "english", "AI", "botprofile", "conversations", "greetings", "health", "science"
]
for corpus in corpus_library:
    trainer.train("chatterbot.corpus." + corpus)